# Cardiovascular Dashboard

#### Environment Setup

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

from holoviews.selection import link_selections
from holoviews import opts


#### Data

In [2]:
df = pd.read_csv('https://chronicdata.cdc.gov/api/views/kztq-p2jf/rows.csv?accessType=DOWNLOAD&api_foundry=true')

pd.set_option('display.max_rows', 20)  
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

df = df.dropna(subset = ['Data_Value'])
df

,row_id,Year,LocationAbbr,LocationDesc,DataSource,PriorityArea1,PriorityArea2,PriorityArea3,PriorityArea4,Category,Topic,Indicator,Data_Value_Type,Data_Value_Unit,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,LowConfidenceLimit,HighConfidenceLimit,Break_Out_Category,Break_Out,CategoryId,TopicId,IndicatorID,Data_Value_TypeID,BreakOutCategoryId,BreakOutId,LocationID,GeoLocation
0,NVSS~2003~32~NV005~AGE08~Crude,2003,NV,Nevada,NVSS,None,None,None,None,Cardiovascular Diseases,Heart Failure,Rate of heart failure mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",333.7,333.7,NaN,NaN,333.6,333.8,Age,75+,C1,T5,NV005,Crude,BOC03,AGE08,32,POINT (-117.07184056399967 39.493240390000494)
1,NVSS~2007~59~NV001~AGE08~Crude,2007,US,United States,NVSS,None,None,None,None,Cardiovascular Diseases,Major Cardiovascular Disease,Rate of major cardiovascular disease mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",2933.6,2933.6,NaN,NaN,2933.6,2933.6,Age,75+,C1,T1,NV001,Crude,BOC03,AGE08,59,NaN
2,NVSS~2012~59~NV001~RAC07~Crude,2012,US,United States,NVSS,None,None,None,None,Cardiovascular Diseases,Major Cardiovascular Disease,Rate of major cardiovascular disease mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",138.2,138.2,NaN,NaN,138.2,138.2,Race,Other,C1,T1,NV001,Crude,BOC04,RAC07,59,NaN
3,NVSS~2011~59~NV001~GEN02~Crude,2011,US,United States,NVSS,None,None,None,None,Cardiovascular Diseases,Major Cardiovascular Disease,Rate of major cardiovascular disease mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",322.2,322.2,NaN,NaN,322.2,322.2,Gender,Female,C1,T1,NV001,Crude,BOC02,GEN02,59,NaN
4,NVSS~2010~59~NV001~AGE05~Crude,2010,US,United States,NVSS,None,None,None,None,Cardiovascular Diseases,Major Cardiovascular Disease,Rate of major cardiovascular disease mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",157.8,157.8,NaN,NaN,157.8,157.8,Age,45-64,C1,T1,NV001,Crude,BOC03,AGE05,59,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158075,NVSS~2012~15~NV008~GEN01~Crude,2012,HI,Hawaii,NVSS,None,None,None,None,Cardiovascular Diseases,Stroke,Rate of hemorrhagic stroke mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",17.0,17.0,NaN,NaN,16.8,20.8,Gender,Male,C1,T6,NV008,Crude,BOC02,GEN01,15,POINT (-157.85774940299973 21.304850435000446)
158076,NVSS~2015~06~NV008~AGE04~Crude,2015,CA,California,NVSS,None,None,None,None,Cardiovascular Diseases,Stroke,Rate of hemorrhagic stroke mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",2.0,2.0,NaN,NaN,1.9,2.1,Age,25-44,C1,T6,NV008,Crude,BOC03,AGE04,6,POINT (-120.99999953799971 37.63864012300047)
158077,NVSS~2013~06~NV008~RAC01~Age-Standardized,2013,CA,California,NVSS,None,None,None,None,Cardiovascular Diseases,Stroke,Rate of hemorrhagic stroke mortality among US adults (18+); NVSS,Age-Standardized,"Rate per 100,000",10.0,10.0,NaN,NaN,10.0,10.0,Race,Non-Hispanic White,C1,T6,NV008,AgeStdz,BOC04,RAC01,6,POINT (-120.99999953799971 37.63864012300047)
158078,NVSS~2012~19~NV008~GEN01~Crude,2012,IA,Iowa,NVSS,None,None,None,None,Cardiovascular Diseases,Stroke,Rate of hemorrhagic stroke mortality among US adults (18+); NVSS,Crude,"Rate per 100,000",11.2,11.2,NaN,NaN,11.0,11.4,Gender,Male,C1,T6,NV008,Crude,BOC02,GEN01,19,POINT (-93.81649055599968 42.46940091300047)


In [3]:
df['Break_Out_Category'].unique()

array(['Age', 'Race', 'Gender', 'Overall'], dtype=object)

In [4]:
race = df[(df['Break_Out_Category'] == 'Race')]
race['Break_Out'].unique()

array(['Other', 'Non-Hispanic Black', 'Hispanic', 'Non-Hispanic White'],
      dtype=object)

In [5]:
disease = df.replace(['Diseases of the Heart (Heart Disease)','Acute Myocardial Infarction (Heart Attack)'], ['Diseases of the Heart','Acute Myocardial Infarction'], inplace=True)
race = df.replace(['Non-Hispanic Black','Non-Hispanic White'], ['Black','White'], inplace=True)

df['Topic'].unique()

array(['Heart Failure', 'Major Cardiovascular Disease', 'Stroke',
       'Diseases of the Heart', 'Coronary Heart Disease',
       'Acute Myocardial Infarction'], dtype=object)

In [6]:
df['Break_Out'].unique()

array(['75+', 'Other', 'Female', '45-64', '25-44', '65+', '35+', '18-24',
       'Male', 'Black', 'Hispanic', 'Overall', 'White'], dtype=object)

In [7]:
idf = df.interactive()

#### Widgets

In [8]:
menu_items =list(df['LocationDesc'].unique())
menu_items.remove('United States')
menu_items.remove('Washington, DC')
menu_items.sort()

state_select = pn.widgets.Select(name='State', options=menu_items)
state_select

Select(name='State', options=['Alabama', 'Alaska', ...], value='Alabama')

In [9]:
Year=list(df['Year'].unique())
Year.sort()
year_select = pn.widgets.Select(name='Year', options=Year)
year_select

Select(name='Year', options=[2000, 2001, 2002, ...], value=2000)

In [10]:
disease_states = df['Topic'].unique()
disease_items = list(disease_states)
disease_select = pn.widgets.Select(name='Disease', options=disease_items)
disease_select

Select(name='Disease', options=['Heart Failure', ...], value='Heart Failure')

#### Disease State Timeline

In [11]:
disease_pipeline = (
    idf[
        (idf.Year <= 2018) &
        (idf.Topic.isin(disease_states)) &
        (idf.LocationDesc == 'United States') &
        (idf.Break_Out_Category == 'Overall') &
        (idf.Data_Value_TypeID == 'AgeStdz')
    ]
    .groupby(['Topic','Year'])['Data_Value'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [12]:
disease_pipeline

In [13]:
disease_state_timeline = disease_pipeline.hvplot(frame_width=800,
                                                 frame_height=175,
                                                 x='Year',
                                                 y='Data_Value',
                                                 by='Topic',
                                                 ylabel='Incidence Rate Per 100,000',
                                                 line_width=1.5,
                                                 color=['pink','red','crimson','indianred','firebrick','darkred'],
                                                 title="CV Disease in USA (2000-2018)")
disease_state_timeline

In [14]:
age = df[(df['Break_Out_Category'] == 'Age')]
age = list(age['Break_Out'].unique())

#### Disease Rate by State Bar Chart

In [15]:
disease_states_bar_pipeline = (
    idf[
        (idf.LocationDesc == state_select) &
        (idf.Topic.isin(disease_states)) &
        (idf.Break_Out_Category == 'Overall') &
        (idf.Data_Value_TypeID == 'AgeStdz')
    ]
    .groupby(['LocationDesc','Topic'])['Data_Value'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='LocationDesc')
    .reset_index(drop=True)
)

In [16]:
disease_states_bar_pipeline

In [17]:
disease_bar_chart = disease_states_bar_pipeline.hvplot(kind='bar',
                                                       frame_width=800,
                                                       frame_height=175,
                                                       x='LocationDesc',
                                                       y='Data_Value',
                                                       by='Topic',
                                                       xlabel='',
                                                       ylabel='Incidence Rate Per 100,000',
                                                       title="Average Rates By State (2000-2018)")
disease_bar_chart = disease_bar_chart.opts(cmap='reds', shared_axes=False)
disease_bar_chart

#### Gender Bar Chart

In [18]:
gender = df[(df['Break_Out_Category'] == 'Gender')]
gender = list(gender['Break_Out'].unique())
gender

['Female', 'Male']

In [19]:
gender_pipeline = (
    idf[
        (idf.Year == year_select) &
        (idf.LocationDesc == state_select) &
        (idf.Topic == disease_select) &
        (idf.Break_Out.isin(gender)) &
        (idf.Data_Value_TypeID == 'AgeStdz')
    ]
    .groupby(['Year','LocationDesc','Topic','Break_Out'])['Data_Value'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='LocationDesc')
    .reset_index(drop=True)
)

In [20]:
gender_pipeline

In [21]:
gender_bar_chart = gender_pipeline.hvplot.barh(
                                          frame_width=300,
                                          frame_height=175,
                                          x='Break_Out',
                                          y='Data_Value',
                                          color='Data_Value',
                                          cmap=['maroon','lavenderblush'],
                                          alpha=0.85,
                                          xlabel='Gender',
                                          ylabel='Incidence Rate Per 100,000',
                                          title="Average Rates By Gender")
gender_bar_chart

#### Age Bar Chart

In [22]:
age_pipeline = (
    idf[
        (idf.Year == year_select) &
        (idf.LocationDesc == state_select) &
        (idf.Topic == disease_select) &
        (idf.Break_Out.isin(age))
    ]
    .groupby(['Year','LocationDesc','Topic','Break_Out'])['Data_Value'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by=['Year','LocationDesc','Break_Out'])
    .reset_index(drop=True)
)

In [23]:
age_pipeline

In [24]:
age_bar_chart = age_pipeline.hvplot.barh(
                                          frame_width=300,
                                          frame_height=175,
                                          x='Break_Out',
                                          y='Data_Value',
                                          color='Data_Value',
                                          cmap='reds',
                                          alpha=0.75,
                                          xlabel='Age',
                                          ylabel='Incidence Rate Per 100,000',
                                          title="Average Rates By Age")
age_bar_chart

#### Race Bar Chart

In [25]:
race = ['Other', 'Black', 'Hispanic', 'White']

race_pipeline = (
    idf[
        (idf.Year == year_select) & 
        (idf.LocationDesc == state_select) & 
        (idf.Topic == disease_select) &
        (idf.Break_Out.isin(race)) &
        (idf.Data_Value_TypeID == 'AgeStdz')
    ]
    .groupby(['Year','LocationDesc','Topic','Break_Out'])['Data_Value'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by=['Year','LocationDesc','Break_Out'])
    .reset_index(drop=True)
)

In [26]:
race_pipeline

In [27]:
race_bar_chart = race_pipeline.hvplot.barh(
                                      frame_width=300,
                                      frame_height=175,
                                      x='Break_Out',
                                      y='Data_Value',
                                      color='Data_Value',
                                      cmap='reds',
                                      xlabel='Race',
                                      ylabel='Incidence Rate Per 100,000',
                                      title="Average Rates By Race")
race_bar_chart

#### Dashboard Creation

In [28]:
link = link_selections.instance()

linked = link(gender_bar_chart + age_bar_chart + race_bar_chart).opts(shared_axes=False)
linked

In [29]:
widgets = pn.WidgetBox(year_select, state_select, disease_select, margin=5)

template = pn.template.FastListTemplate(
    title='Cardiovascular Disease in USA',
    sidebar=[pn.pane.Markdown('## Visualization of incidence rates of cardiovascular diseases that occurred in the United States from 2000-2018. Data retrieved from National Vital Statistics System.'),
             pn.pane.Markdown('## Settings'),
             widgets,
             pn.pane.Markdown('###### Dashboard created by Thanh-Trang Tran')],
    main=[pn.Row(pn.Column(disease_state_timeline.panel(width=1000))),
          pn.Row(pn.Column(disease_bar_chart.panel(width=1000))),
          pn.Row(pn.Column(linked.panel(width=370)))],
    header_background='#8B0A50',
)

#template.show()
template.servable();

Launching server at http://localhost:50470
